## Problem 1: Baseline Probability Calculation

In [2]:
import numpy as np
import pandas as pd

# Shared transition matrix and state setup
P = np.array([
    [0.99, 0.01, 0.0],
    [0.0, 0.99, 0.01],
    [0.0, 0.0, 1.0]
])
pi_0 = np.array([1.0, 0.0, 0.0])
states = [1, 2, 3]

def compute_state_prob(pi_0, P, t):
    return pi_0 @ np.linalg.matrix_power(P, t)

def lognormal_emission(y, s, lambda_dict, zeta):
    mu = lambda_dict[s]
    sigma = zeta
    return (1 / (y * sigma * np.sqrt(2 * np.pi))) * np.exp(-((np.log(y) - mu) ** 2) / (2 * sigma ** 2))

def hmm_forward(measurements, lambda_dict, zeta):
    T = len(measurements)
    n_states = len(states)
    alpha = np.zeros((T, n_states))
    for j in range(n_states):
        alpha[0, j] = pi_0[j] * lognormal_emission(measurements[0], states[j], lambda_dict, zeta)
    alpha[0] /= np.sum(alpha[0])
    for t in range(1, T):
        for j in range(n_states):
            alpha[t, j] = lognormal_emission(measurements[t], states[j], lambda_dict, zeta) * np.sum(alpha[t-1] * P[:, j])
        alpha[t] /= np.sum(alpha[t])
    return alpha


In [3]:
pi_50 = compute_state_prob(pi_0, P, 50)
pi_100 = compute_state_prob(pi_0, P, 100)

print("Problem 1:")
print(f"t = 50: Intact={pi_50[0]:.4f}, Damage={pi_50[1]:.4f}, Failure={pi_50[2]:.4f}")
print(f"t = 100: Intact={pi_100[0]:.4f}, Damage={pi_100[1]:.4f}, Failure={pi_100[2]:.4f}")


Problem 1:
t = 50: Intact=0.6050, Damage=0.3056, Failure=0.0894
t = 100: Intact=0.3660, Damage=0.3697, Failure=0.2642


## Problem 2: Posterior Probability with Observations

In [4]:
df = pd.read_csv("Measures.csv", header=None)
measurements = df[0].values

lambda_values_informative = {1: 0.5, 2: 0.75, 3: 1.0}
zeta = 0.8

alpha_informative = hmm_forward(measurements, lambda_values_informative, zeta)
pi_50 = alpha_informative[-1]
pi_100 = compute_state_prob(pi_50, P, 50)

print("\nProblem 2:")
print(f"Posterior at t=50: Intact={pi_50[0]:.4f}, Damage={pi_50[1]:.4f}, Failure={pi_50[2]:.4f}")
print(f"Predicted at t=100: Intact={pi_100[0]:.4f}, Damage={pi_100[1]:.4f}, Failure={pi_100[2]:.4f}")



Problem 2:
Posterior at t=50: Intact=0.5744, Damage=0.3883, Failure=0.0373
Predicted at t=100: Intact=0.3475, Damage=0.4104, Failure=0.2421


## Problem 3: Effect of Changed Emission Parameters

In [5]:
lambda_values_uniform = {1: 1.0, 2: 1.0, 3: 1.0}

alpha_uniform = hmm_forward(measurements, lambda_values_uniform, zeta)
pi_50 = alpha_uniform[-1]
pi_100 = compute_state_prob(pi_50, P, 50)

print("\nProblem 3:")
print(f"Posterior at t=50: Intact={pi_50[0]:.4f}, Damage={pi_50[1]:.4f}, Failure={pi_50[2]:.4f}")
print(f"Predicted at t=100: Intact={pi_100[0]:.4f}, Damage={pi_100[1]:.4f}, Failure={pi_100[2]:.4f}")



Problem 3:
Posterior at t=50: Intact=0.6111, Damage=0.3025, Failure=0.0864
Predicted at t=100: Intact=0.3697, Damage=0.3697, Failure=0.2605
